In [2]:
import pickle as pkl
from transformer_raw.modules import encoder, decoder
def load(path : str) -> None:
	pickle_encoder = open(f'{path}/encoder.pkl', 'rb')
	pickle_decoder = open(f'{path}/decoder.pkl', 'rb')
	# print(type(self.encoder.dropout.scale))

	encoder = pkl.load(pickle_encoder)
	decoder = pkl.load(pickle_decoder)
	pickle_encoder.close()
	pickle_decoder.close()
	return encoder, decoder


In [3]:
path = "./saved_models/seq2seq_model/10"
encoder, decoder = load(path)


In [13]:
import cupy as np
a = np.random.rand(5, 10, 20)
b = np.random.rand(20, 30)
c = np.random.rand(10, 30)


In [14]:
from transformer.utils import Utils
import tenseal as ts
utils = Utils("./keys")


In [15]:
a_ = utils.encrypt_matrix(a)
b_ = utils.encrypt_matrix(b)
c_ = utils.encrypt_matrix(c)


In [16]:
a_T = a_.transpose()
mean = utils.mean(a_T, axis=0)
var = utils.var(a_T, axis=0)

In [33]:
from typing import Union
def x_power_n(X: ts.CKKSTensor, n : Union[int, float] , mean : float = None) -> ts.CKKSTensor:
	# Approximating Square Roots with Taylor Series
	if type(n) is int and n > 0:
		return X ** n
	else:
		assert mean is not None, "Mean must be provided for non-integer powers"
		def fx(x : float) -> float:
			return pow(x, n)
		def fx_(x : float) -> float:
			return n * pow(x, n - 1)
		def fx__(x : float) -> float:
			return n*(n-1) * pow(x, n-2)
		return fx(mean) + fx_(mean) * (X - mean) + 1/2 * fx__(mean) * (X - mean) ** 2
x_centered = a_T - mean
stddev_inv = x_power_n(var + 0.0001, -0.5, mean=1.0)
X_hat = (x_centered * stddev_inv).transpose()
print(X_hat)

In [41]:
gamma = utils.encrypt_matrix(np.random.rand(20))
beta = utils.encrypt_matrix(np.random.rand(20))
gamma.shape
# output_data = gamma * X_hat + beta
# print(output_data.shape)


[20]

In [ ]:

# encoder.layers[1].self_attention_norm.mean
for a in dir(encoder.layers[1].self_attention_norm):
    print(a, type(getattr(encoder.layers[1].self_attention_norm, a)))

X_centered <class 'cupy.ndarray'>
X_hat <class 'cupy.ndarray'>
X_hat_T <class 'cupy.ndarray'>
__class__ <class 'type'>
__delattr__ <class 'method-wrapper'>
__dict__ <class 'dict'>
__dir__ <class 'builtin_function_or_method'>
__doc__ <class 'str'>
__eq__ <class 'method-wrapper'>
__format__ <class 'builtin_function_or_method'>
__ge__ <class 'method-wrapper'>
__getattribute__ <class 'method-wrapper'>
__gt__ <class 'method-wrapper'>
__hash__ <class 'method-wrapper'>
__init__ <class 'method'>
__init_subclass__ <class 'builtin_function_or_method'>
__le__ <class 'method-wrapper'>
__lt__ <class 'method-wrapper'>
__module__ <class 'str'>
__ne__ <class 'method-wrapper'>
__new__ <class 'builtin_function_or_method'>
__reduce__ <class 'builtin_function_or_method'>
__reduce_ex__ <class 'builtin_function_or_method'>
__repr__ <class 'method-wrapper'>
__setattr__ <class 'method-wrapper'>
__sizeof__ <class 'builtin_function_or_method'>
__str__ <class 'method-wrapper'>
__subclasshook__ <class 'builtin_fu

In [39]:
encoder.layers[1].self_attention_norm.gamma.size

256